# DataWorkshop Master Class
## Adrian Skutnik

*Miejsce 4. z wynikiem:* **2.14155**

# 1. Krótko o sobie

* 4 lata w Santander Bank Polska 
    * Analityk danych w Pionie Zarządzania Ryzykiem. 
    * Głównie SQL, MS Excel, Jupyter Notebook jako wizualizacja danych.
* Wokół ML krążę, zarówno prywatnie, jak i zawodowo, ale na co dzień nie modeluję :(

# 2. Jak się za to zabrać? 

![](3_pliki.png)

**1. Preprocessing danych** – przetworzenie zmiennych słownikowych, wektorowych na odrębne kolumny.

**2. Feature engineering**  – Tu działo się najwięcej:

> * Zamiana zmiennych powierzchni na float64
> * najpierw zwykły factorize zmiennych kategorycznych ...
> * … a następnie szeregowanie zmiennych kategorycznych

![](factorize.png) 

In [1]:
def replace_factor_by_stat(df, char_name, stat_name, price):
    "Zastępuję losowy faktor statystyką dla danej cechy"
    
    price_stat = df[price].agg(stat_name)
    
    keys = pd.pivot_table(df, index=[char_name], values=[price], aggfunc=[stat_name]).index
    values = pd.pivot_table(df, index=[char_name], values=[price], aggfunc=[stat_name])[(stat_name,price)]
    char_stat_map = defaultdict(lambda: price_stat,zip(keys, values))
    
    df['{}_repl_{}'.format(char_name, stat_name)] = df[char_name].apply(lambda x: char_stat_map[x]) 
    print('New column in df: {}_repl_{}'.format(char_name, stat_name))
    
    return df   

* **Wreszcie, największy postęp to modelowanie zmiennej cena za metr, a nie samej ceny!**

![](cena.png)

![](cena_za_metr.png)

**3. Modelowanie**

> * Dodatkowo zlogarytmowanie zmiennej price_per_m2
> * Dodanie zmiennych losowych i sprzątanie zbędnych zmiennych

![](log_cena_za_metr.png)

In [ ]:
for i in range(5):
    df['random_{}'.format(i)] = np.random.normal(size=len(df))
    feats.append('random_{}'.format(i))   

> Ramka na wejściu zawierała **219 zmiennych**    
> Modelowanie ostatecznie na **83 zmiennych**

In [2]:
# (2.355716375929465, 0.233763690588422) <-- submit 3 (1.88070)
# (2.2888983069605917, 0.27275301838603083) <-- submit 4 (1.86007)
# (2.10085497423071, 0.29354647610120993) <-- submit 5 (1.71501)

# xgb_params={
#   'n_estimators': 350,
#   'colsample_bytree': 0.3,
#   'learning_rate': 0.05,
#   'max_depth': 15,
#   'min_child_weight': 3,
#   'subsample': 0.75}

# (2.031405362730201, 0.27745594834270715) <-- submit 6 (1.68999) oczyszczenie ze zmiennych losowych

# xgb_params={
#   'n_estimators': 400,
#   'colsample_bytree': 0.3,
#   'learning_rate': 0.05,
#   'max_depth': 15,
#   'min_child_weight': 3,
#   'subsample': 0.75}

# (2.026766325034033, 0.2786812280083919) <-- submit 7 (1.68381) best na ręcznym wyborze parametrów


# xgb_params={
#     'n_estimators': 350, 
#     'colsample_bytree': 0.3, 
#     'learning_rate': 0.075, 
#     'max_depth': 15, 
#     'min_child_weight': 6, 
#     'subsample': 0.85}


# (2.0658578335869375, 0.31226221389120995) <-- submit 8 (1.70633) najlepszy z Random Search


# xgb_params={
#     'n_estimators': 375, 
#     'colsample_bytree': 0.5, 
#     'learning_rate': 0.1, 
#     'max_depth': 15, 
#     'min_child_weight': 8, 
#     'subsample': 0.95}

# (2.0983975078759416, 0.27381624010553185) <-- submit 9 (1.70633) (prawie) najstabilniejszy z Random Search

# 4 Bolączki!

> * Brak czasu :( Przy modelowaniu zawsze można znaleźć jeszcze lepsze ścieżki
> * Dziury w pamięci (od podstawowych funkcji pandas'a do GridSearch'a i hyperopt'a)
> * Szukanie w oczywistościach nie przynosi efektów: data publikacji ogłoszenia

In [3]:
date_map = {'января': 1,
            'февраля':2,
           'марта': 3,
           'апреля': 4,
           'мая': 5,
           'июня': 6,
           'июля':7,
           'августа': 8,
           'сентября': 9,
           'октября': 10,
           'ноября': 11,
           'декабря': 12}

In [ ]:
df['is_today'] = df['Дата публикации:'].map(lambda x: 'сегодня' in x).map(int)
df['is_yesterday'] = df['Дата публикации:'].map(lambda x: 'вчера' in x).map(int)

df['today_time'] = df.loc[df['is_today']==1, 'Дата публикации:'].map(lambda x: datetime.strptime(x[-5:], '%H:%M'))
df['yesterday_time'] = df.loc[df['is_yesterday']==1, 'Дата публикации:'].map(lambda x: datetime.strptime(x[-5:], '%H:%M'))

df.loc[df['is_today']==1, 'Дата публикации:']  = ''
df.loc[df['is_yesterday']==1, 'Дата публикации:']  = ''

def convert_date(date_str): 
    
    date_lst = date_str.split(' ')
    
    if len(date_lst) == 0:
        date = ''
    
    
    if len(date_lst) == 2:
        date_lst[0] = date_lst[0]
        date_lst[1] = str(date_map[date_lst[1]])
        date_lst.append('2019')
        
        date = datetime.strptime('/'.join(date_lst), '%d/%m/%Y')
        return date
 
        
        
    if len(date_lst) == 3:
        date_lst[0] = date_lst[0]
        date_lst[1] = str(date_map[date_lst[1]])
                
        date = datetime.strptime('/'.join(date_lst), '%d/%m/%Y')
    
    

convert_date(df['Дата публикации:'][4])

df['publish_date'].max()

pd.Timestamp('2019-05-18 00:00:00')

df.loc[df['is_today']==1, 'publish_date'] = pd.Timestamp('2019-05-20 00:00:00')
df.loc[df['is_yesterday']==1, 'publish_date'] = pd.Timestamp('2019-05-19 00:00:00')

def convert_date_to_days(publish_date):
    
    try:
        return (date(2019,5,20) - publish_date.date()).days
    except:
        return -1
        
df['publish_days'] = df['publish_date'].map(convert_date_to_days)

![](cena_od_dnia.png)

# 5 Delta rozwoju

> * Porządkowanie i jakość kodu - nic odkrywczego, a jednak...
> * Wiedza i doświadczenie w domenie jednak się przydaje
> * Feature engineering ma moc! :) 

# Dziękuję!!! 